# Final project: Identification of vulnerable population groups

## 1. Problem statement

According to a December 2021 [inFOM survey](https://www.cbr.ru/Collection/Collection/File/39633/inFOM_21-12.pdf), 27% of Russians have enough money only for food, and another 9% cannot afford a nutritious diet. These people are especially attentive to prices, and the rate of growth of food prices usually exceeds the average rate of inflation. At the same time, Rosstat believes that food expenses should make up approximately 36% of a Russian's average monthly expenses (another 10% goes to utilities and housing, 4% goes to medicines). 

Until 2021, the "poverty line" (living below the subsistence minimum) in Russia was determined by the cost of the [minimum food basket](https://base.garant.ru/70306880/). In the same year, the government "untied" the poverty level from the prices of basic products: since 2021, the subsistence minimum is calculated as 44.2% of the median income of Russian citizens for the previous year.

You have at your disposal data on income, morbidity, socially vulnerable groups of the Russian population and other economic and demographic data.

Your task as a data scientist:
* cluster the regions of Russia and determine which of them are in the greatest need of assistance to low-income/disadvantaged segments of the population;
* describe the population groups facing poverty;
* determine:
    * whether the number of children, pensioners and other socially vulnerable groups affects the poverty level in the region;
    * whether the level of poverty/social disadvantage is related to production and consumption in the region;
    * what other dependencies can be observed in relation to socially vulnerable segments of the population.

## 2. Getting to know the data

### 2.0 Import dependencies and define helper code

#### 2.0.1 Import dependencies

In [1]:
import pandas as pd

#### 2.0.2 Define helper code

In [41]:
def rename_columns(df, prefix):
    dict_columns = {}
    for i in range(1, df.shape[1]):
        dict_columns[df.columns[i]] = prefix + df.columns[i]

    return df.rename(columns=dict_columns)

### 2.1 Population data

Data source contains the population dynamics of the Russian regions across the years 1999 and 2022. Columns:
1. region - name of the region of the Russian Federation. Data for districts uniting multiple regions has been removed.
2. population_1999-population_2022 - columns containing the population count for each of the reagions in the corresponding year.

In [48]:
data_population = pd.read_excel('data/population.xlsx', 'report')
for i in range(data_population.shape[0]//2):
    data_population.iloc[i, 1:] = data_population.iloc[i+1, 1:]
    data_population.drop(index=i+1, inplace=True)
    data_population = data_population.reset_index(drop=True)

data_population = rename_columns(data_population, 'population_')
data_population.head()

,region,population_1999,population_2000,population_2001,population_2002,population_2003,population_2004,population_2005,population_2006,population_2007,...,population_2013,population_2014,population_2015,population_2016,population_2017,population_2018,population_2019,population_2020,population_2021,population_2022
0,Белгородская область,1494868.0,1501699.0,1506976.0,1508137.0,1511899.0,1513860.0,1511662.0,1511715.0,1514153.0,...,1540985.0,1544108.0,1547936.0,1550137.0,1552865.0,1549876.0,1547418.0,1549151.0,1541259.0,1531917.0
1,Брянская область,1437471.0,1423178.0,1407965.0,1391430.0,1375004.0,1360249.0,1344132.0,1327652.0,1312748.0,...,1253666.0,1242599.0,1232940.0,1225741.0,1220530.0,1210982.0,1200187.0,1192491.0,1182682.0,1168771.0
2,Владимирская область,1592184.0,1575507.0,1558052.0,1539179.0,1520057.0,1509571.0,1497598.0,1486453.0,1475861.0,...,1421742.0,1413321.0,1405613.0,1397168.0,1389599.0,1378337.0,1365805.0,1358416.0,1342099.0,1323659.0
3,Воронежская область,2458558.0,2441337.0,2422371.0,2397111.0,2374461.0,2367457.0,2364932.0,2360912.0,2353805.0,...,2330377.0,2328959.0,2331147.0,2333477.0,2335408.0,2333768.0,2327821.0,2324205.0,2305608.0,2287678.0
4,Ивановская область,1210603.0,1194595.0,1178969.0,1161861.0,1144540.0,1131027.0,1116739.0,1101862.0,1089837.0,...,1048961.0,1043130.0,1036909.0,1029838.0,1023170.0,1014646.0,1004180.0,997135.0,987032.0,976918.0


### 2.2 Child mortality in rural areas

Data source contains child mortality in rural areas on the first life year in persons per Russian region across the years 1990 and 2021. Columns:
1. region - name of the region of the Russian Federation. Data for districts uniting multiple regions has been removed.
2. child_mortality_rural_1990-child_mortality_rural_2021 - columns containing the child mortality in rural areas for each of the reagions in the corresponding year.

In [ ]:
data_child_mortality_rural = pd.read_excel('data/child_mortality_rural_1990_2021.xlsx')
data_child_mortality_rural = rename_columns(data_child_mortality_rural, 'child_mortality_rural_')
data_child_mortality_rural.head()

,child_mortality,child_mortality_rural_1990,child_mortality_rural_1991,child_mortality_rural_1992,child_mortality_rural_1993,child_mortality_rural_1994,child_mortality_rural_1995,child_mortality_rural_1996,child_mortality_rural_1997,child_mortality_rural_1998,...,child_mortality_rural_2012,child_mortality_rural_2013,child_mortality_rural_2014,child_mortality_rural_2015,child_mortality_rural_2016,child_mortality_rural_2017,child_mortality_rural_2018,child_mortality_rural_2019,child_mortality_rural_2020,child_mortality_rural_2021
0,Белгородская область,103.0,92.0,75.0,79.0,80.0,72.0,72.0,67.0,61.0,...,43.0,48.0,41.0,42.0,36.0,34.0,33.0,16.0,22.0,20.0
1,Брянская область,124.0,109.0,83.0,121.0,99.0,104.0,96.0,67.0,75.0,...,46.0,47.0,39.0,44.0,36.0,31.0,12.0,11.0,12.0,13.0
2,Владимирская область,80.0,58.0,60.0,62.0,46.0,50.0,47.0,38.0,39.0,...,30.0,31.0,23.0,31.0,28.0,17.0,22.0,15.0,10.0,14.0
3,Воронежская область,138.0,179.0,156.0,149.0,154.0,137.0,133.0,132.0,125.0,...,32.0,33.0,33.0,25.0,24.0,18.0,22.0,7.0,12.0,10.0
4,Ивановская область,74.0,44.0,40.0,57.0,50.0,41.0,31.0,39.0,27.0,...,13.0,19.0,10.0,16.0,15.0,4.0,7.0,9.0,3.0,4.0


### 2.3 Child mortality in urban areas

Data source contains child mortality in urban areas on the first life year in persons per Russian region across the years 1990 and 2021. Columns:
1. region - name of the region of the Russian Federation. Data for districts uniting multiple regions has been removed.
2. child_mortality_urban_1990-child_mortality_urban_2021 - columns containing the child mortality in urban areas for each of the reagions in the corresponding year.

In [46]:
data_child_mortality_urban = pd.read_excel('data/child_mortality_urban_1990_2021.xlsx')
data_child_mortality_urban = rename_columns(data_child_mortality_urban, 'child_mortality_urban_')
data_child_mortality_urban.head()

,region,child_mortality_urban_1990,child_mortality_urban_1991,child_mortality_urban_1992,child_mortality_urban_1993,child_mortality_urban_1994,child_mortality_urban_1995,child_mortality_urban_1996,child_mortality_urban_1997,child_mortality_urban_1998,...,child_mortality_urban_2012,child_mortality_urban_2013,child_mortality_urban_2014,child_mortality_urban_2015,child_mortality_urban_2016,child_mortality_urban_2017,child_mortality_urban_2018,child_mortality_urban_2019,child_mortality_urban_2020,child_mortality_urban_2021
0,Белгородская область,209.0,198.0,165.0,165.0,153.0,131.0,102.0,100.0,99.0,...,84.0,68.0,62.0,68.0,72.0,43.0,40.0,23.0,25.0,34.0
1,Брянская область,198.0,195.0,200.0,176.0,157.0,125.0,116.0,135.0,107.0,...,81.0,77.0,87.0,67.0,67.0,65.0,33.0,26.0,29.0,14.0
2,Владимирская область,221.0,209.0,179.0,148.0,165.0,146.0,114.0,123.0,130.0,...,97.0,86.0,87.0,76.0,72.0,59.0,45.0,43.0,51.0,52.0
3,Воронежская область,261.0,262.0,262.0,213.0,194.0,184.0,140.0,143.0,177.0,...,134.0,153.0,112.0,101.0,96.0,90.0,77.0,74.0,58.0,73.0
4,Ивановская область,181.0,187.0,174.0,168.0,142.0,141.0,128.0,127.0,110.0,...,58.0,67.0,58.0,48.0,51.0,31.0,28.0,31.0,25.0,20.0
